# TranVanDat_18020295_HumanReid 

In [2]:
! git clone https://github.com/KaiyangZhou/deep-person-reid.git

Cloning into 'deep-person-reid'...
remote: Enumerating objects: 9809, done.
remote: Counting objects: 100% (283/283), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 9809 (delta 188), reused 267 (delta 183), pack-reused 9526
Receiving objects: 100% (9809/9809), 9.59 MiB | 17.98 MiB/s, done.
Resolving deltas: 100% (7226/7226), done.


In [3]:
!ls

deep-person-reid  sample_data


In [4]:
cd deep-person-reid/

/content/deep-person-reid


In [5]:
! pip install -r requirements.txt

     |████████████████████████████████| 5.8 MB 9.4 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 185 kB 46.1 MB/s 
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 69 kB 8.1 MB/s 
     |████████████████████████████████| 64 kB 2.7 MB/s 
     |████████████████████████████████| 73 kB 1.8 MB/s 
     |████████████████████████████████| 68 kB 6.7 MB/s 
     |████████████████████████████████| 41 kB 647 kB/s 


## resnet50

### khai báo thư viện

In [6]:
import torchreid

/content/deep-person-reid/torchreid/metrics/rank.py:12: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  'Cython evaluation (very fast so highly recommended) is '


### Load data manager

kích thước ảnh trong data 256 x 128 số lượng ảnh là 12936

In [7]:
# Load data
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='market1501',
    targets='market1501',
    height=256,
    width=128,
    batch_size_train=32,
    batch_size_test=100,
    transforms=['random_flip', 'random_crop']
)

Building train transforms ...
+ resize to 256x128
+ random flip
+ random crop (enlarge to 288x144 and crop 256x128)
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
Creating directory "/content/deep-person-reid/reid-data/market1501"
* url="http://188.138.127.15:81/Datasets/Market-1501-v15.09.15.zip"
* destination="/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
...100%, 145 MB, 3767 KB/s, 39 seconds passed
Extracting "/content/deep-person-reid/reid-data/market1501/Market-1501-v15.09.15.zip"
Market1501 dataset is ready
=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    | 

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


=> Loaded Market1501
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |   751 |    12936 |         6
  query    |   750 |     3368 |         6
  gallery  |   751 |    15913 |         6
  ----------------------------------------


  **************** Summary ****************
  source            : ['market1501']
  # source datasets : 1
  # source ids      : 751
  # source images   : 12936
  # source cameras  : 6
  target            : ['market1501']
  *****************************************




### Build model sử dụng model (resnet50)

nhận dạng nhiều lớp sử dụng loss là softmax nhận dạng 751 lớp 

In [20]:
num = datamanager.num_train_pids
num

751

In [ ]:
model_Resnet50 = torchreid.models.build_model(
    name='resnet50',
    num_classes=num,
    loss='softmax',
    pretrained=True
)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
model_Resnet50 = model_Resnet50.cuda()

dùng hàm tối ưu là adam

In [ ]:
optimizer = torchreid.optim.build_optimizer(
        model_Resnet50,
        optim='adam',
        lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=10 # bước nhảy là 10
)

### Train model

In [ ]:
# Build engine
engine = torchreid.engine.ImageSoftmaxEngine(
      datamanager,
      model_Resnet50,
      optimizer=optimizer,
      scheduler=scheduler,
      label_smooth=True
)
# Train
engine.run(
    save_dir='log/resnet50',
    max_epoch=15, # train 15 epoch
    eval_freq=10,
    print_freq=200, # chạy mỗi lần 200 / 404 
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/15][200/404]	time 0.435 (0.445)	data 0.001 (0.003)	eta 0:43:25	loss 1.2922 (1.3218)	acc 96.8750 (97.4375)	lr 0.000030
epoch: [1/15][400/404]	time 0.441 (0.443)	data 0.000 (0.002)	eta 0:41:47	loss 1.3098 (1.2966)	acc 96.8750 (97.7656)	lr 0.000030
epoch: [2/15][200/404]	time 0.441 (0.444)	data 0.001 (0.004)	eta 0:40:23	loss 1.2658 (1.2394)	acc 100.0000 (98.9375)	lr 0.000030
epoch: [2/15][400/404]	time 0.441 (0.443)	data 0.001 (0.002)	eta 0:38:48	loss 1.2051 (1.2359)	acc 96.8750 (98.8984)	lr 0.000030
epoch: [3/15][200/404]	time 0.438 (0.445)	data 0.000 (0.004)	eta 0:37:26	loss 1.3621 (1.2124)	acc 93.7500 (99.0938)	lr 0.000030
epoch: [3/15][400/404]	time 0.438 (0.443)	data 0.000 (0.002)	eta 0:35:49	loss 1.3002 (1.2140)	acc 96.8750 (99.0859)	lr 0.000030
epoch: [4/15][200/404]	time 0.444 (0.445)	data 0.001 (0.004)	eta 0:34:27	loss 1.2017 (1.1986)	acc 100.0000 (99.3750)	lr 0.000030
epoch: [4/15][400/404]	time 0.443 (0.443)	data 0.001 (0.003)	eta 0:32:50	loss 1.2234 (1.2031)	acc 100.

## osnet_ain_x0_25

### Build model sử dụng model (osnet_ain_x0_25)

nhận dạng nhiều lớp sử dụng loss là softmax nhận dạng 751 lớp 

In [ ]:
num = datamanager.num_train_pids
num

751

In [ ]:
model_osnet_ain_x0_25 = torchreid.models.build_model(
    name='osnet_ain_x0_25',
    num_classes=num,
    loss='softmax',
    pretrained=True
)

Downloading...
From: https://drive.google.com/uc?id=1SxQt2AvmEcgWNhaRb2xC4rP6ZwVDP0Wt
To: /root/.cache/torch/checkpoints/osnet_ain_x0_25_imagenet.pth
100%|██████████| 3.08M/3.08M [00:00<00:00, 81.7MB/s]

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x0_25_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
model_osnet_ain_x0_25 = model_osnet_ain_x0_25.cuda()

dùng hàm tối ưu là adam

In [ ]:
optimizer = torchreid.optim.build_optimizer(
        model_osnet_ain_x0_25,
        optim='adam',
        lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=10 # bước nhảy là 10
)

### Train model

In [ ]:
# Build engine
engine = torchreid.engine.ImageSoftmaxEngine(
      datamanager,
      model_osnet_ain_x0_25,
      optimizer=optimizer,
      scheduler=scheduler,
      label_smooth=True
)
# Train
engine.run(
    save_dir='log/osnet_ain_x0_25',
    max_epoch=15, # train 15 epoch
    eval_freq=10,
    print_freq=200, # chạy mỗi lần 200 / 404 
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/15][200/404]	time 0.185 (0.199)	data 0.000 (0.004)	eta 0:19:25	loss 4.7945 (5.7500)	acc 25.0000 (10.5938)	lr 0.000300
epoch: [1/15][400/404]	time 0.160 (0.196)	data 0.000 (0.002)	eta 0:18:31	loss 3.5532 (4.8909)	acc 56.2500 (24.0156)	lr 0.000300
epoch: [2/15][200/404]	time 0.196 (0.199)	data 0.001 (0.003)	eta 0:18:03	loss 2.4849 (2.6801)	acc 71.8750 (69.4219)	lr 0.000300
epoch: [2/15][400/404]	time 0.162 (0.195)	data 0.000 (0.002)	eta 0:17:07	loss 2.0403 (2.4487)	acc 84.3750 (74.3516)	lr 0.000300
epoch: [3/15][200/404]	time 0.185 (0.198)	data 0.000 (0.003)	eta 0:16:37	loss 1.5891 (1.7682)	acc 96.8750 (90.3281)	lr 0.000300
epoch: [3/15][400/404]	time 0.162 (0.196)	data 0.000 (0.002)	eta 0:15:51	loss 1.6138 (1.7283)	acc 90.6250 (90.7734)	lr 0.000300
epoch: [4/15][200/404]	time 0.184 (0.198)	data 0.001 (0.003)	eta 0:15:22	loss 1.4186 (1.4835)	acc 100.0000 (96.2812)	lr 0.000300
epoch: [4/15][400/404]	time 0.162 (0.195)	data 0.000 (0.002)	eta 0:14:27	loss 1.5354 (1.4908)	acc 96.87

## osnet_ain_x1_0

### Build model sử dụng model (osnet_ain_x1_0)

nhận dạng nhiều lớp sử dụng loss là softmax nhận dạng 751 lớp 

In [ ]:
num = datamanager.num_train_pids
num

751

In [ ]:
model_osnet_ain_x1_0 = torchreid.models.build_model(
    name='osnet_ain_x1_0',
    num_classes=num,
    loss='softmax',
    pretrained=True
)

Downloading...
From: https://drive.google.com/uc?id=1-CaioD9NaqbHK_kzSMW8VE4_3KcsRjEo
To: /root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 130MB/s]


Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ain_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
model_osnet_ain_x1_0 = model_osnet_ain_x1_0.cuda()

dùng hàm tối ưu là adam

In [ ]:
optimizer = torchreid.optim.build_optimizer(
        model_osnet_ain_x1_0,
        optim='adam',
        lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=10 # bước nhảy là 10
)

### Train model

In [ ]:
# Build engine
engine = torchreid.engine.ImageSoftmaxEngine(
      datamanager,
      model_osnet_ain_x1_0,
      optimizer=optimizer,
      scheduler=scheduler,
      label_smooth=True
)

# Train
engine.run(
    save_dir='log/osnet_ain_x1_0',
    max_epoch=15, # train 15 epoch
    eval_freq=10,
    print_freq=200, # chạy mỗi lần 200 / 404 
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/15][200/404]	time 0.486 (0.490)	data 0.001 (0.003)	eta 0:47:51	loss 4.7946 (5.7737)	acc 25.0000 (9.7969)	lr 0.000300
epoch: [1/15][400/404]	time 0.481 (0.488)	data 0.000 (0.002)	eta 0:46:00	loss 3.1229 (4.8238)	acc 59.3750 (25.7109)	lr 0.000300
epoch: [2/15][200/404]	time 0.487 (0.490)	data 0.001 (0.004)	eta 0:44:32	loss 2.1085 (2.4404)	acc 81.2500 (74.2031)	lr 0.000300
epoch: [2/15][400/404]	time 0.482 (0.488)	data 0.001 (0.002)	eta 0:42:43	loss 1.5885 (2.1629)	acc 93.7500 (80.2969)	lr 0.000300
epoch: [3/15][200/404]	time 0.484 (0.489)	data 0.002 (0.004)	eta 0:41:12	loss 1.3595 (1.4821)	acc 96.8750 (94.9844)	lr 0.000300
epoch: [3/15][400/404]	time 0.484 (0.488)	data 0.000 (0.002)	eta 0:39:26	loss 1.3487 (1.4502)	acc 96.8750 (95.4219)	lr 0.000300
epoch: [4/15][200/404]	time 0.486 (0.489)	data 0.002 (0.004)	eta 0:37:52	loss 1.2198 (1.2659)	acc 100.0000 (98.5000)	lr 0.000300
epoch: [4/15][400/404]	time 0.487 (0.487)	data 0.000 (0.002)	eta 0:36:07	loss 1.2556 (1.2672)	acc 100.00

## osnet_ibn_x1_0

### Build model sử dụng model (osnet_ibn_x1_0)

nhận dạng nhiều lớp sử dụng loss là softmax nhận dạng 751 lớp 

In [ ]:
num = datamanager.num_train_pids
num

751

In [ ]:
model_osnet_ibn_x1_0 = torchreid.models.build_model(
    name='osnet_ibn_x1_0',
    num_classes=num,
    loss='softmax',
    pretrained=True
)

Downloading...
From: https://drive.google.com/uc?id=1sr90V6irlYYDd4_4ISU2iruoRG8J__6l
To: /root/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 148MB/s]

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
model_osnet_ibn_x1_0 = model_osnet_ibn_x1_0.cuda()

dùng hàm tối ưu là adam

In [ ]:
optimizer = torchreid.optim.build_optimizer(
        model_osnet_ibn_x1_0,
        optim='adam',
        lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=10 # bước nhảy là 10
)

### Train model

In [ ]:
# Build engine
engine = torchreid.engine.ImageSoftmaxEngine(
      datamanager,
      model_osnet_ibn_x1_0,
      optimizer=optimizer,
      scheduler=scheduler,
      label_smooth=True
)
# Train
engine.run(
    save_dir='log/osnet_ibn_x1_0',
    max_epoch=15, # train 15 epoch
    eval_freq=10,
    print_freq=200, # chạy mỗi lần 200 / 404 
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/15][200/404]	time 0.490 (0.496)	data 0.000 (0.004)	eta 0:48:26	loss 5.1432 (5.9260)	acc 18.7500 (7.7969)	lr 0.000300
epoch: [1/15][400/404]	time 0.485 (0.493)	data 0.000 (0.002)	eta 0:46:32	loss 3.6579 (5.0099)	acc 43.7500 (22.1719)	lr 0.000300
epoch: [2/15][200/404]	time 0.484 (0.496)	data 0.001 (0.004)	eta 0:45:05	loss 2.1382 (2.6203)	acc 81.2500 (70.6875)	lr 0.000300
epoch: [2/15][400/404]	time 0.488 (0.494)	data 0.000 (0.002)	eta 0:43:18	loss 2.0223 (2.3016)	acc 78.1250 (77.5391)	lr 0.000300
epoch: [3/15][200/404]	time 0.498 (0.497)	data 0.001 (0.003)	eta 0:41:48	loss 1.4929 (1.5182)	acc 100.0000 (94.2344)	lr 0.000300
epoch: [3/15][400/404]	time 0.488 (0.495)	data 0.001 (0.002)	eta 0:40:00	loss 1.4888 (1.4830)	acc 93.7500 (94.5781)	lr 0.000300
epoch: [4/15][200/404]	time 0.497 (0.498)	data 0.000 (0.003)	eta 0:38:35	loss 1.2127 (1.2805)	acc 100.0000 (98.4844)	lr 0.000300
epoch: [4/15][400/404]	time 0.485 (0.495)	data 0.000 (0.002)	eta 0:36:43	loss 1.3090 (1.2804)	acc 96.87

## osnet_x0_25

### Build model sử dụng model (osnet_x0_25)

nhận dạng nhiều lớp sử dụng loss là softmax nhận dạng 751 lớp 

In [ ]:
num = datamanager.num_train_pids
num

751

In [ ]:
model_osnet_x0_25 = torchreid.models.build_model(
    name='osnet_x0_25',
    num_classes=num,
    loss='softmax',
    pretrained=True
)

Downloading...
From: https://drive.google.com/uc?id=1rb8UN5ZzPKRc_xvtHlyDh-cSz88YX9hs
To: /root/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth
100%|██████████| 2.97M/2.97M [00:00<00:00, 84.9MB/s]

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x0_25_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
model_osnet_x0_25 = model_osnet_x0_25.cuda()

dùng hàm tối ưu là adam

In [ ]:
optimizer = torchreid.optim.build_optimizer(
        model_osnet_x0_25,
        optim='adam',
        lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=10 # bước nhảy là 10
)

### Train model

In [ ]:
# Build engine
engine = torchreid.engine.ImageSoftmaxEngine(
      datamanager,
      model_osnet_x0_25,
      optimizer=optimizer,
      scheduler=scheduler,
      label_smooth=True
)
# Train
engine.run(
    save_dir='log/osnet_x0_25',
    max_epoch=15, # train 15 epoch
    eval_freq=10,
    print_freq=200, # chạy mỗi lần 200 / 404 
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/15][200/404]	time 0.191 (0.198)	data 0.000 (0.004)	eta 0:19:21	loss 4.7568 (5.8976)	acc 31.2500 (7.7031)	lr 0.000300
epoch: [1/15][400/404]	time 0.162 (0.195)	data 0.000 (0.002)	eta 0:18:24	loss 4.0267 (5.1050)	acc 37.5000 (18.5312)	lr 0.000300
epoch: [2/15][200/404]	time 0.187 (0.194)	data 0.000 (0.003)	eta 0:17:39	loss 2.5451 (3.0144)	acc 81.2500 (62.4219)	lr 0.000300
epoch: [2/15][400/404]	time 0.157 (0.193)	data 0.000 (0.002)	eta 0:16:53	loss 2.1280 (2.7324)	acc 84.3750 (68.1016)	lr 0.000300
epoch: [3/15][200/404]	time 0.190 (0.196)	data 0.000 (0.004)	eta 0:16:32	loss 1.9562 (1.9344)	acc 81.2500 (86.8281)	lr 0.000300
epoch: [3/15][400/404]	time 0.155 (0.194)	data 0.001 (0.002)	eta 0:15:39	loss 1.8728 (1.8564)	acc 81.2500 (88.2344)	lr 0.000300
epoch: [4/15][200/404]	time 0.184 (0.195)	data 0.000 (0.003)	eta 0:15:06	loss 1.5022 (1.5636)	acc 100.0000 (94.7188)	lr 0.000300
epoch: [4/15][400/404]	time 0.155 (0.193)	data 0.000 (0.002)	eta 0:14:16	loss 1.5626 (1.5497)	acc 87.500

## osnet_x1_0

### Build model sử dụng model (osnet_x1_0)

nhận dạng nhiều lớp sử dụng loss là softmax nhận dạng 751 lớp 

In [8]:
num = datamanager.num_train_pids
num

751

In [9]:
model_osnet_x1_0 = torchreid.models.build_model(
    name='osnet_x1_0',
    num_classes=num,
    loss='softmax',
    pretrained=True
)

Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 53.7MB/s]

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [10]:
model_osnet_x1_0 = model_osnet_x1_0.cuda()

dùng hàm tối ưu là adam

In [11]:
optimizer = torchreid.optim.build_optimizer(
        model_osnet_x1_0,
        optim='adam',
        lr=0.0003
)
scheduler = torchreid.optim.build_lr_scheduler(
        optimizer,
        lr_scheduler='single_step',
        stepsize=10 # bước nhảy là 10
)

### Train model

In [12]:
 # Build engine
engine = torchreid.engine.ImageSoftmaxEngine(
      datamanager,
      model_osnet_x1_0,
      optimizer=optimizer,
      scheduler=scheduler,
      label_ smooth=True
)
# Train
engine.run(
    save_dir='log/osnet_x1_0',
    max_epoch=15, # train 15 epoch
    eval_freq=10,
    print_freq=200, # chạy mỗi lần 200 / 404 
    test_only=False
)

=> Start training


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch: [1/15][200/404]	time 0.486 (0.500)	data 0.000 (0.020)	eta 0:48:48	loss 5.0376 (5.9232)	acc 12.5000 (7.9219)	lr 0.000300
epoch: [1/15][400/404]	time 0.495 (0.492)	data 0.000 (0.010)	eta 0:46:22	loss 3.5376 (5.0096)	acc 53.1250 (22.0938)	lr 0.000300
epoch: [2/15][200/404]	time 0.484 (0.488)	data 0.001 (0.004)	eta 0:44:20	loss 2.1724 (2.6284)	acc 84.3750 (69.4375)	lr 0.000300
epoch: [2/15][400/404]	time 0.484 (0.486)	data 0.000 (0.002)	eta 0:42:35	loss 1.8218 (2.3124)	acc 84.3750 (76.5547)	lr 0.000300
epoch: [3/15][200/404]	time 0.484 (0.488)	data 0.000 (0.004)	eta 0:41:03	loss 1.4465 (1.5290)	acc 96.8750 (93.9375)	lr 0.000300
epoch: [3/15][400/404]	time 0.485 (0.486)	data 0.000 (0.002)	eta 0:39:18	loss 1.3959 (1.4821)	acc 93.7500 (94.5469)	lr 0.000300
epoch: [4/15][200/404]	time 0.480 (0.487)	data 0.000 (0.003)	eta 0:37:42	loss 1.2323 (1.2771)	acc 96.8750 (98.0781)	lr 0.000300
epoch: [4/15][400/404]	time 0.481 (0.485)	data 0.000 (0.002)	eta 0:35:59	loss 1.4022 (1.2761)	acc 87.5000